# Example code that accompanies the publication: Integrating criticality concepts into road network disruption assessments for volcanic eruptions

This python notebook presents example code that describes the analysis undertaken to compare road disruption from the Merapi 2010 and Kelud 2014 eruptions, on Java, Indonesia. A paper is currently being prepared and reviewed that describes the methodology and conceptual framework. In this notebook, we focus our attention on the how the criticality dataset from this paper is generated. 

In [1]:
# Import the necessary functions and libraries

from RNDS_functions import prepare_roads
from RNDS_functions import Assign_road_hierarchy
from RNDS_functions import Assign_strategic_nodes
from RNDS_functions import Assign_community_facilities
from RNDS_functions import Assign_criticality
from RNDS_functions import Get_LoR_Score
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display

In [2]:
# Assign directories

print("Assigning directories")
dir = os.path.abspath('')
roads_dir = os.path.join(dir, 'Geospatial_data/roads')
POIs_dir = os.path.join(dir, 'Geospatial_data/POI')
Strategic_nodes_dir = os.path.join(dir, "Geospatial_data/Strategic_nodes")

Assigning directories


## Data
We first read in the geospatial data. There are libraries (e.g. ) that will be able pull this data and filter it appropriately. In the interests of computational efficiency in the jupyter notebook, we will just use the pre-processed roads, points of interest, and strategic nodes data. See the paper for where these datasets where obtained from. 

In [3]:
# Read in the data

print("reading in roads")
roads = gpd.read_file(roads_dir + "/Java_roads.shp")
print("reading in Points of Interest")
POIs = gpd.read_file(POIs_dir + "/filtered_POIs.gpkg")
print("reading in strategic nodes")
Strategic_nodes = gpd.read_file(Strategic_nodes_dir + "/Strategic_nodes.gpkg")



reading in roads
reading in Points of Interest


C:\Users\hayes.jlee\AppData\Local\Continuum\anaconda3\envs\ox\lib\site-packages\geopandas\geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


reading in strategic nodes


## Prepare roads data

The OSM roads data has a lot of unnecessary information associated with it. So, here we remove what we don't need. We also add a "Road_ID" field to give each road segment it's own unique identifier. This is used later during attribute joins. 

In [4]:
# Prepare roads file

roads = prepare_roads(roads)

Preparing roads file


C:\Users\hayes.jlee\AppData\Local\Continuum\anaconda3\envs\ox\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


# Assigning road hierarchy scores

This function assigns a score to each road segment based on the road hierarcy scoring system. 

In [5]:
# Assign road hierarchy
roads = Assign_road_hierarchy(roads)
# Showing a subset as an example

display(roads[['name', 'highway', 'Road_ID', 'hierarchy']])

Assigning road hierarchy scores to road network
Assigning road hierarchy scores to road network is complete


,name,highway,Road_ID,hierarchy
0,None,unclassified,0,1.0
1,Jalan Wonolelo - Pleret,secondary,1,2.0
2,None,secondary,2,2.0
3,None,unclassified,3,1.0
4,Jalan Banyakan 1,unclassified,4,1.0
...,...,...,...,...
2137129,Jalan Husein Sastranegara,secondary,2137129,2.0
2137130,Jalan Husein Sastranegara,secondary,2137130,2.0
2137131,None,living_street,2137131,1.0
2137132,None,living_street,2137132,1.0


In [6]:
# Assign strategic nodes values

roads_SN = Assign_strategic_nodes(roads, Strategic_nodes)

# Displaying output
roads_SN_non_zero = roads_SN.loc[roads_SN['point'] > 0]
display(roads_SN_non_zero[['name', 'highway', 'Road_ID', 'hierarchy', 'point', 'Node score']])

Reprojecting road network and strategic nodes to pseudo mercator
Setting spatial index
Finding closest road to each strategic node
Snapping nodes to nearest road


third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r must be Point
third argument of GEOSProject_r mu

Counting number of snapped nodes touching each road
Assigning strategic node score
Strategic node score successfully applied


,name,highway,Road_ID,hierarchy,point,Node score
23567,None,service,23567,1.0,1,4.0
38666,None,residential,38666,1.0,1,4.0
84421,None,service,84421,1.0,2,8.0
107676,None,service,107676,1.0,1,4.0
147254,Jalan Jenderal Achmad Yani Timur,unclassified,147254,1.0,1,4.0
...,...,...,...,...,...,...
2051146,None,service,2051146,1.0,1,4.0
2051337,None,service,2051337,1.0,1,4.0
2051349,None,service,2051349,1.0,2,8.0
2055579,None,service,2055579,1.0,1,4.0


In [7]:
# Assign community facilities values

roads_SN_POI = Assign_community_facilities(POIs, roads, roads_SN)


# Displaying output
roads_SN_POI_non_zero = roads_SN_POI.loc[roads_SN_POI['Priority'] > 0]
display(roads_SN_POI_non_zero[['name', 'highway', 'Road_ID', 'hierarchy', 'point', 'Node score', 'Priority']])

Assigning access to community services and facilities score
Setting priority scores for each point of interest
Reprojecting POI to pseudo mercator
Setting spatial index
Finding closest road to each POI
Snapping nodes to nearest road
Summing priority scores for each road
merging roads with summed POI


,name,highway,Road_ID,hierarchy,point,Node score,Priority
1,Jalan Wonolelo - Pleret,secondary,1,2.0,0,0.0,0.1
38,None,tertiary,38,2.0,0,0.0,0.1
48,None,residential,48,1.0,0,0.0,2.0
2359,None,residential,2359,1.0,0,0.0,0.1
2901,None,residential,2901,1.0,0,0.0,0.1
...,...,...,...,...,...,...,...
2136889,Jalan Pool PPD,unclassified,2136889,1.0,0,0.0,2.0
2136925,Jalan Gerbang Poris Indah,residential,2136925,1.0,0,0.0,18.0
2137040,None,residential,2137040,1.0,0,0.0,3.0
2137045,None,residential,2137045,1.0,0,0.0,3.0


In [8]:
# Assign road criticality

roads_criticality = Assign_criticality(roads_SN_POI)


# Displaying output
#roads_criticality = roads_criticality.loc[roads_SN_POI['Priority'] > 0]
display(roads_criticality[['name', 'highway', 'Road_ID', 'hierarchy', 'point', 'Node score', 'Priority', 'Criticality score']])

Merging road networks
Calculating criticality
Assigning criticality score


,name,highway,Road_ID,hierarchy,point,Node score,Priority,Criticality score
0,None,unclassified,0,1.0,0,0.0,0.0,1.0
1,Jalan Wonolelo - Pleret,secondary,1,2.0,0,0.0,0.1,100.0
2,None,secondary,2,2.0,0,0.0,0.0,100.0
3,None,unclassified,3,1.0,0,0.0,0.0,1.0
4,Jalan Banyakan 1,unclassified,4,1.0,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
2137129,Jalan Husein Sastranegara,secondary,2137129,2.0,0,0.0,0.0,100.0
2137130,Jalan Husein Sastranegara,secondary,2137130,2.0,0,0.0,0.0,100.0
2137131,None,living_street,2137131,1.0,0,0.0,0.0,1.0
2137132,None,living_street,2137132,1.0,0,0.0,0.0,1.0


In [9]:
# Assign length of road score

roads_criticality_LoR = Get_LoR_Score(roads_criticality=roads_criticality)

# Displaying output
#roads_criticality = roads_criticality.loc[roads_SN_POI['Priority'] > 0]
display(roads_criticality_LoR[['name', 'highway', 'Road_ID', 'hierarchy', 'point', 'Node score', 'Priority', 'Criticality score', 'length', 'LoR score']])

reprojecting roads layer
Calculating segment lengths
Obtaining quantiles
Assigning length of road score


,name,highway,Road_ID,hierarchy,point,Node score,Priority,Criticality score,length,LoR score
0,None,unclassified,0,1.0,0,0.0,0.0,1.0,2490.116444,10.0
1,Jalan Wonolelo - Pleret,secondary,1,2.0,0,0.0,0.1,100.0,4928.821655,10.0
2,None,secondary,2,2.0,0,0.0,0.0,100.0,2529.998094,10.0
3,None,unclassified,3,1.0,0,0.0,0.0,1.0,993.398539,10.0
4,Jalan Banyakan 1,unclassified,4,1.0,0,0.0,0.0,1.0,1729.538954,10.0
...,...,...,...,...,...,...,...,...,...,...
2137129,Jalan Husein Sastranegara,secondary,2137129,2.0,0,0.0,0.0,100.0,146.340357,1.0
2137130,Jalan Husein Sastranegara,secondary,2137130,2.0,0,0.0,0.0,100.0,60.804111,0.1
2137131,None,living_street,2137131,1.0,0,0.0,0.0,1.0,84.433606,0.1
2137132,None,living_street,2137132,1.0,0,0.0,0.0,1.0,116.555925,1.0


The file that has been created through the above process now contains an attribute for criticality (Criticality score) and length of road (LoR score). The next step is to use this file and assign impact scores. This can be done by either creating a function that will loop through different scenarios or by simply saving the file and manually assigning impact scores within a GIS software (e.g. Qgis) if there are not too many scenarios to consider. Then, by multiplying the Criticality score, LoR score, and the Impact score, a Road Segment Disruption Score (RSDS) is obtained that quantifies the disruption (in a relative sense) for any given road segment. Summing the RSDS across all affected road segments will then give a Road Network Disruption Score (RNDS), which gives an indiciation of the overall network disruption to the entire road network by a given scenario. The RNDS scores can then be compared across different scenarios, allowing one to compare and rank the potential disruption of different scenarios. 